# RAWG Data Engineering Project - Final Orchestrator
This notebook runs the full ETL pipeline from data ingestion (Bronze) to transformation (Silver).

In [1]:
import logging
import os
from datetime import datetime, timedelta
from src.ingestor import GameDataIngestor
from src.transformer import GameTransformer
from deltalake import DeltaTable

# --- LOGGING SETUP ---
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, "pipeline.log")

# Clear existing handlers to prevent duplicates on re-run
logger = logging.getLogger()
if logger.hasHandlers():
    logger.handlers.clear()

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, mode='a', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logging.info(f"Logging iniciado. Archivo: {log_file}")

# --- PIPELINE CONFIGURATION ---
# Calculate pipeline window (e.g., last 30 days or custom backfill)
end_date = datetime.now()
start_date = end_date - timedelta(days=30)
str_start = start_date.strftime('%Y-%m-%d')
str_end = end_date.strftime('%Y-%m-%d')

print(f"Pipeline Window: {str_start} to {str_end}")

2025-12-08 23:28:41,558 - INFO - Logging iniciado. Archivo: logs\pipeline.log


Pipeline Window: 2025-11-08 to 2025-12-08


In [2]:
# --- PART 1: INGESTION (BRONZE LAYER) ---
ingestor = GameDataIngestor()

print("1. Starting Full Load for Genres...")
try:
    ingestor.get_genres_full()
except Exception as e:
    print(f"Genre load failed: {e}")

print("\n2. Starting Incremental Load for Games...")
try:
    ingestor.get_games_incremental(start_date=str_start, end_date=str_end)
except Exception as e:
    print(f"Game load failed: {e}")

2025-12-08 23:28:41,564 - INFO - Iniciando Full Load para genres...


1. Starting Full Load for Genres...


2025-12-08 23:28:42,741 - INFO - Guardados 19 géneros en c:\Users\cyber\OneDrive\Desktop\Curso UTN\TP-games\data\bronze\genres (Mode: overwrite).
2025-12-08 23:28:42,742 - INFO - Iniciando Carga Incremental para juegos (2025-11-08 a 2025-12-08)...
2025-12-08 23:28:42,743 - INFO - Cargando página 1...



2. Starting Incremental Load for Games...


2025-12-08 23:28:43,897 - INFO - Cargando página 2...
2025-12-08 23:28:45,229 - INFO - Cargando página 3...
2025-12-08 23:28:46,562 - INFO - Cargando página 4...
2025-12-08 23:28:47,930 - INFO - Cargando página 5...
2025-12-08 23:28:49,095 - INFO - Cargando página 6...
2025-12-08 23:28:50,197 - INFO - Cargando página 7...
2025-12-08 23:28:51,503 - INFO - Cargando página 8...
2025-12-08 23:28:52,673 - INFO - Cargando página 9...
2025-12-08 23:28:54,326 - INFO - Cargando página 10...
2025-12-08 23:28:55,739 - INFO - Cargando página 11...
2025-12-08 23:28:56,991 - INFO - Cargando página 12...
2025-12-08 23:28:58,225 - INFO - Cargando página 13...
2025-12-08 23:28:59,378 - INFO - Cargando página 14...
2025-12-08 23:29:00,549 - INFO - Cargando página 15...
2025-12-08 23:29:01,966 - INFO - Cargando página 16...
2025-12-08 23:29:03,185 - INFO - Cargando página 17...
2025-12-08 23:29:07,261 - INFO - Cargando página 18...
2025-12-08 23:29:09,121 - INFO - Cargando página 19...
2025-12-08 23:29:1

In [3]:
# --- PART 2: TRANSFORMATION (SILVER LAYER) ---
print("\n3. Starting Transformation Process...")
transformer = GameTransformer()
transformer.process()

2025-12-08 23:29:59,566 - INFO - Starting Silver Layer transformation...



3. Starting Transformation Process...


2025-12-08 23:29:59,982 - INFO - Loaded 1044 records from Bronze Games.
2025-12-08 23:30:00,007 - INFO - Saved 1044 records to c:\Users\cyber\OneDrive\Desktop\Curso UTN\TP-games\data\silver\games_refined
2025-12-08 23:30:00,028 - INFO - Saved 15 analytics records to c:\Users\cyber\OneDrive\Desktop\Curso UTN\TP-games\data\silver\games_analytics


In [4]:
# --- VERIFICATION ---
from src.config import Config
import os
import pandas as pd

silver_path = Config.SILVER_PATH
games_refined_path = os.path.join(silver_path, "games_refined")

print(f"\nChecking Silver Table at: {games_refined_path}")
if os.path.exists(games_refined_path):
    try:
        dt = DeltaTable(games_refined_path)
        df_silver = dt.to_pandas()
        print("Silver Dataframe Head:")
        display(df_silver.head())
        print(f"Total Records: {len(df_silver)}")
    except Exception as e:
        print(f"Could not read Silver table: {e}")
else:
    print("Silver table not found. Pipeline might have failed or no data was fetched.")


Checking Silver Table at: c:\Users\cyber\OneDrive\Desktop\Curso UTN\TP-games\data\silver\games_refined
Silver Dataframe Head:


,id,slug,name,released,released_year,tba,background_image,rating,rating_top,metacritic,is_top_rated,primary_genre,extraction_date
0,455518,skate-story,Skate Story,2025-12-08,2025,False,https://media.rawg.io/media/games/53e/53e81bf3...,0.0,0,NaN,False,Action,2025-12-08
1,1015351,temple-of-the-green-moon-prologue,Temple of the Green Moon: Prologue,2025-12-05,2025,False,https://media.rawg.io/media/screenshots/5d7/5d...,0.0,0,NaN,False,Adventure,2025-12-08
2,1015352,hot-asses,HOT ASSES,2025-12-05,2025,False,https://media.rawg.io/media/screenshots/39a/39...,0.0,0,NaN,False,Adventure,2025-12-08
3,41626,routine,Routine,2025-12-04,2025,False,https://media.rawg.io/media/games/c2c/c2c572ae...,0.0,0,NaN,False,Adventure,2025-12-08
4,1015350,fight-for-america,Fight For America,2025-12-04,2025,False,https://media.rawg.io/media/screenshots/c79/c7...,0.0,0,NaN,False,Strategy,2025-12-08


Total Records: 1044
